# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PIVResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353).

## Preparation and Imports

In [1]:
from ontolutils.ex import dcat, m4i, schema, prov
from pivmetalib import PIVMETA, pivmeta, sd

from ssnolib import StandardNameTable

We will describe variables using standard names. For this we, decide to use [this Standard Name table](https://zenodo.org/records/15297431)

In [2]:
dist = dcat.Distribution(
    downloadURL='https://zenodo.org/records/17550472/files/Standard_Name_Table_for_Particle_Image_Velocimetry.jsonld',
    mediaType='application/json+ld'
)
dowloaded_filename = dist.download()

In [3]:
dowloaded_filename

WindowsPath('Standard_Name_Table_for_Particle_Image_Velocimetry.jsonld')

In [4]:
snt = StandardNameTable.parse(dowloaded_filename, fmt='jsonld')

In [5]:
standard_names = snt.get_standard_names_as_frozen_dataclass()

In [6]:
result_dist = pivmeta.ImageVelocimetryDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat',
    hasPIVDataType=PIVMETA.ResultData
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [7]:
piv_image_dist = pivmeta.ImageVelocimetryDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    hasPIVDataType=PIVMETA.ExperimentalImage
)
piv_image_dist

ImageVelocimetryDistribution(id=_:N262c4d8bbec9485caf56a38490bf5086, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)

In [8]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(id=_:N97007b0faf9e4433a1d2371cc57655ff, title=ILA Vortex Pair, distribution=[ImageVelocimetryDistribution(id=_:N400cdc70975d46398d8b554f6bf28810, title=Result File, downloadURL=file:///vp1a.dat, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ResultData), ImageVelocimetryDistribution(id=_:N262c4d8bbec9485caf56a38490bf5086, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)])

Before defining the PIV processing steps, we need to describe the software used:

In [9]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

## Processing steps

A `PIVProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [10]:
pre = pivmeta.PIVPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            parameter=m4i.NumericalVariable(
                name='rotation',
                hasNumericalValue=180,
                unit='deg',
                quantity_kind='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=m4i.NumericalVariable(
                standard_name=standard_names.image_filter_kernel_size,
                hasNumericalValue=3,
            )
        )
    ]
)
pre

PIVPreProcessing(id=_:N2bbab470665840e7bf176aa67553a710, realizesMethod=[Method(id=_:N5fb708a7c7d546478a64500d8c706120, description=Rotates the input image by 180 deg, parameter=NumericalVariable(id=_:Nc71c61cf5b2e42b3b786673c33774788, hasNumericalValue=180.0, name=rotation, unit=deg, quantity_kind=https://qudt.org/vocab/unit/DEG)), Method(id=_:N14a6b22181634faeb599af214d16e926, description=Applies a median filter to the image, hasParameter=NumericalVariable(id=_:Nb17a9129b88d4082842267556a219e7c, hasNumericalValue=3.0, standard_name=image_filter_kernel_size))], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PIVValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [11]:
calgo = pivmeta.CorrelationMethod(
    name='Standard (FFT) Correlation',
    hasWindowWeightingFunction=PIVMETA.SquareWindow,
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
print(calgo.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"
    },
    "@type": "pivmeta:CorrelationMethod",
    "schema:description": "The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).",
    "pivmeta:hasWindowWeightingFunction": {
        "@id": "https://matthiasprobst.github.io/pivmeta#SquareWindow"
    },
    "name": "Standard (FFT) Correlation",
    "@id": "_:Nf80363af716b44b0a6eb8908650394dd"
}


### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [12]:
v = m4i.NumericalVariable(
    standard_name=standard_names.x_initial_interrogation_window_size,
    hasNumericalValue=64,
)
print(v.model_dump_jsonld(resolve_keys=False))

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"
    },
    "@type": "m4i:NumericalVariable",
    "m4i:hasNumericalValue": 64.0,
    "@id": "_:N2a06d64c57974a979925dc9ac30cccd6"
}


In [13]:
#print(v.dump_jsonld())

In [14]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        m4i.NumericalVariable(
            standard_name=standard_names.x_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            description='Number of multigrid passes',
            hasNumericalValue=3,
        )
    ]
)
int_meth.hasParameter[-1]

NumericalVariable(id=_:Nf8918c9d18dc4976b254787eb1231624, hasNumericalValue=3.0, description=Number of multigrid passes)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [15]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        hasNumericalValue=3.0
    )
)

In [16]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            hasNumericalValue=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            hasNumericalValue=1.0
        )
    ]
)

In [17]:
proc = pivmeta.PIVEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PIVEvaluation(id=_:N24b6e2ce987f4879a3db9d1e621b8a3c, label=piv evaluation, realizesMethod=[CorrelationMethod(id=_:Nf80363af716b44b0a6eb8908650394dd, description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., hasWindowWeightingFunction=https://matthiasprobst.github.io/pivmeta#SquareWindow, name=Standard (FFT) Correlation), Method(id=_:Nc38c4f7ef9c940aaaff3d28f87722c74, description=Run a multigrid PIV algorithm on all images, name=Multigrid interrogation method, hasParameter=[NumericalVariable(id=_:N7e9ed70af099464e870b61e2231afc95, hasNumericalValue=64.0, standard_name=x_initial_interrogation_window_size), NumericalVariable(id=_:Nd540d63f5ddd4bc3862c5a1bb0e8e56d, hasNumericalValue=64.0, standard_name=y_initial_interrogation_window_size), NumericalVariable(id=_:N9007ad87d5294578b1fa2d3bb0c9d2be, hasNumericalValue=16.0, standard_name=x_final_interrogation_window_size), NumericalVariable(id=_:N18d3f3dd326a4edcbe19930e95546b98, hasNumericalValue=16.0, standard_name=y_final_interrogation_window_size), NumericalVariable(id=_:N58a618b98e674f56b67924bc9fd2a1b2, hasNumericalValue=8.0, standard_name=x_final_interrogation_window_overlap_size), NumericalVariable(id=_:N610825c1f14e468fb6ac220e95f9a10d, hasNumericalValue=8.0, standard_name=y_final_interrogation_window_overlap_size), NumericalVariable(id=_:Nf8918c9d18dc4976b254787eb1231624, hasNumericalValue=3.0, description=Number of multigrid passes)]), OutlierDetectionMethod(id=_:Naf264d1aee214aba85370c9ee84e62bd, name=normalized median test, hasParameter=NumericalVariable(id=_:Nd45418c12c304337944839100899ea5f, label=threshold, hasNumericalValue=3.0)), OutlierDetectionMethod(id=_:Nb3e08222d6c04a69be49a823f2015d9d, name=dynamic mean test, hasParameter=[NumericalVariable(id=_:Nda930ef306cd472ebf7bdbea98827cd6, label=mean, hasNumericalValue=2.0), NumericalVariable(id=_:N565204b488a641bab65ad0e811c794d4, label=var, hasNumericalValue=1.0)])])

In [18]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    parameter=m4i.NumericalVariable(label='kernel', hasNumericalValue=2.0)
)

In [19]:
post = pivmeta.PIVPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
post.model_dump_jsonld()         

'{\n    "@context": {\n        "owl": "http://www.w3.org/2002/07/owl#",\n        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n        "dcterms": "http://purl.org/dc/terms/",\n        "schema": "https://schema.org/",\n        "skos": "http://www.w3.org/2004/02/skos/core#",\n        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",\n        "obo": "http://purl.obolibrary.org/obo/",\n        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",\n        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#"\n    },\n    "@type": "pivmeta:PIVProcessingStep",\n    "rdfs:label": "Post processing",\n    "m4i:realizesMethod": {\n        "@type": "m4i:Method",\n        "schema:description": "applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.",\n        "m4i:hasParameter": {\n            "@type": "m4i:NumericalVariable",\n            "rdfs:label": "kernel",\n            "m4i:hasNumericalValue": 2.0,\n            "@id": "_

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [20]:
post.hasOutput = ds

## 4. dump PIV run to JSON-LD

In [21]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

In [22]:
print(pre.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "pivmeta:PIVPostProcessing",
    "m4i:realizesMethod": [
        {
            "@type": "m4i:Method",
            "schema:description": "Rotates the input image by 180 deg",
            "m4i:hasParameter": {
                "@type": "m4i:NumericalVariable",
                "m4i:hasNumericalValue": 180.0,
                "n

In [23]:
with open('piv_process.json', 'w') as f:
    f.write(proc.model_dump_jsonld())

In [24]:
print(piv.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "pims": "http://www.molmod.info/semantics/pims-ii.ttl#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "prov": "http://www.w3.org/ns/prov#"
    },
    "@type": "m4i:ProcessingStep",
    "rdfs:label": "PIV Run",
    "startsWith": {
        "@type": "pivmeta:PIVPostProcessing",
        "m4i:realizesMethod": [
            {
                "@type": "m4i:Method",
                "schema:description": "Rotates the input image by 180 deg",
                "m4i:hasParameter": {
        